In [ ]:
  # import libraries
try:
    # %tensorflow_version only exists in Colab.
    !pip install tf-nightly
except Exception:
    pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2.12.0-dev20221230


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-12-30 15:15:36--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.2’

train-data.tsv.2    100%[===================>] 349.84K  1.18MB/s    in 0.3s    

2022-12-30 15:15:36 (1.18 MB/s) - ‘train-data.tsv.2’ saved [358233/358233]

--2022-12-30 15:15:37--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.2’

valid-data.tsv.2    100%[====

In [ ]:
traindata = pd.read_table(train_file_path)
vailddata = pd.read_table(test_file_path)

In [ ]:
vailddata.head()

,ham,i am in hospital da. . i will return home in evening
0,ham,"not much, just some textin'. how bout you?"
1,ham,i probably won't eat at all today. i think i'm...
2,ham,don‘t give a flying monkeys wot they think and...
3,ham,who are you seeing?
4,ham,your opinion about me? 1. over 2. jada 3. kusr...


In [ ]:
#might be needed to append this into pd dataframe

In [ ]:
traindata.columns = ['Classification', 'message']
vailddata.columns = ['Classification', 'message']

traindata.head()

,Classification,message
0,ham,you can never do nothing
1,ham,"now u sound like manky scouse boy steve,like! ..."
2,ham,mum say we wan to go then go... then she can s...
3,ham,never y lei... i v lazy... got wat? dat day ü ...
4,ham,in xam hall boy asked girl tell me the startin...


In [ ]:
fulldata = traindata.message.append(vailddata.message, ignore_index=True)


In [ ]:
a = fulldata.str.split().apply(len)

In [ ]:

print(a.mean()) #mean amount of words in each row

15.586819895852038


In [ ]:
print(len(fulldata))

5569


In [ ]:
from keras.preprocessing.text import Tokenizer

# Initialize the Tokenizer
tokenizer = Tokenizer()

# Fit the Tokenizer on the entire data so it has all the words in vocab
tokenizer.fit_on_texts(fulldata)

# Convert the training data to sequences of tokens
train_X = tokenizer.texts_to_sequences(traindata.message)

# Convert the validation data to sequences of tokens
val_X = tokenizer.texts_to_sequences(vailddata.message)

In [ ]:
print(val_X[0])
print(vailddata.message.loc[0])

[25, 122, 37, 116, 7819, 50, 742, 3]
not much, just some textin'. how bout you?


In [ ]:

train_X = keras.preprocessing.sequence.pad_sequences(train_X, maxlen=16, padding='post', truncating='post')
val_X =  keras.preprocessing.sequence.pad_sequences(val_X, maxlen=16, padding='post', truncating='post')

In [ ]:
print(val_X)

[[  25  122   37 ...    0    0    0]
 [   1  388  569 ...   60   13  414]
 [1683  138    4 ...  107  277   15]
 ...
 [ 131   22    3 ... 1325    2   13]
 [  33  158  856 ...   68  158    2]
 [  25  958   47 ...    2  829  248]]


In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

8995


In [ ]:

train_Y = traindata.Classification

val_Y = vailddata.Classification

In [ ]:
val_Y.replace(to_replace = ['ham','spam'], value = [0,1],inplace = True)
train_Y.replace(to_replace = ['ham','spam'], value = [0,1], inplace = True)
train_Y.head()

0    0
1    0
2    0
3    0
4    0
Name: Classification, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(train_X, train_Y, random_state=2, test_size = 0.1)

In [ ]:
print(len(train_X))
print(len(test_X))
print(len(val_X))

3760
418
1391


In [ ]:
model = keras.Sequential([
    
  keras.layers.Embedding(input_dim=vocab_size, output_dim=75, input_length=16),
  keras.layers.LSTM(units=256, dropout=0.3, recurrent_dropout=0.3),
  keras.layers.Dense(units=1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.001),metrics=[ 'acc'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 16, 75)            674625    
                                                                 
 lstm_4 (LSTM)               (None, 256)               339968    
                                                                 
 dense_4 (Dense)             (None, 1)                 257       
                                                                 
Total params: 1,014,850
Trainable params: 1,014,850
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_X, train_Y, epochs=10, batch_size = 32,validation_data=(val_X, val_Y))

Epoch 1/10
118/118 [==============================] - 30s 224ms/step - loss: 0.1710 - acc: 0.9383 - val_loss: 0.0856 - val_acc: 0.9763
Epoch 2/10
118/118 [==============================] - 11s 95ms/step - loss: 0.0458 - acc: 0.9870 - val_loss: 0.0691 - val_acc: 0.9792
Epoch 3/10
118/118 [==============================] - 10s 84ms/step - loss: 0.0189 - acc: 0.9968 - val_loss: 0.0994 - val_acc: 0.9770
Epoch 4/10
118/118 [==============================] - 11s 90ms/step - loss: 0.0071 - acc: 0.9987 - val_loss: 0.1382 - val_acc: 0.9741
Epoch 5/10
118/118 [==============================] - 10s 82ms/step - loss: 0.0152 - acc: 0.9963 - val_loss: 0.0743 - val_acc: 0.9813
Epoch 6/10
118/118 [==============================] - 10s 81ms/step - loss: 0.0109 - acc: 0.9971 - val_loss: 0.0716 - val_acc: 0.9784
Epoch 7/10
118/118 [==============================] - 10s 84ms/step - loss: 0.0020 - acc: 0.9997 - val_loss: 0.0935 - val_acc: 0.9813
Epoch 8/10
118/118 [==============================] - 11s 89m

In [ ]:
y_pred = model.predict(test_X)
y_prediction = []
for i in y_pred:
  if i > 0.3:
    y_prediction.append(1)
  else:
    y_prediction.append(0)


14/14 [==============================] - 0s 6ms/step


In [ ]:
print(y_prediction)
print(test_Y)

[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

# Calculate the precision and recall for the model
precision, recall, f1_score, support = precision_recall_fscore_support(test_Y, y_prediction, average='binary')

print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')

F1 = 2 * (precision * recall) / (precision + recall)
print(f'F1 score the (higher the better between 0 and 1) is:  {f1_score:.3f}')


Precision: 0.948
Recall: 0.948
F1 score the (higher the better between 0 and 1) is:  0.948


In [ ]:

pred_text = ["how are you doing today?"]
pred_text  = tokenizer.texts_to_sequences(pred_text)

pred_text = [[elem for sublist in pred_text for elem in sublist]]
print(pred_text)
pred_text = keras.preprocessing.sequence.pad_sequences(pred_text, maxlen=16, padding='post', truncating='post')
pred_text = pred_text.flatten()
print(pred_text)

[[50, 22, 3, 159, 89]]
[ 50  22   3 159  89   0   0   0   0   0   0   0   0   0   0   0]


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text  = tokenizer.texts_to_sequences([pred_text])

  pred_text = [[elem for sublist in pred_text for elem in sublist]]
  print(pred_text)
  pred_text = keras.preprocessing.sequence.pad_sequences(pred_text, maxlen=16, padding='post', truncating='post')
  #pred_text = pred_text.flatten()
  print(pred_text)

  prediction = model.predict(pred_text)
  if prediction[0][0] > 0.3:
    label = 'spam'
  else:
    label = 'ham'
  print(prediction[0][0], label)
  return (prediction[0][0], label)

pred_text = "how are you doing today"

prediction = predict_message(pred_text)
print(prediction)

[[50, 22, 3, 159, 89]]
[[ 50  22   3 159  89   0   0   0   0   0   0   0   0   0   0   0]]
1/1 [==============================] - 0s 24ms/step
2.920014e-07 ham
(2.920014e-07, 'ham')


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


[[50, 22, 3, 159, 89]]
[[ 50  22   3 159  89   0   0   0   0   0   0   0   0   0   0   0]]
1/1 [==============================] - 0s 25ms/step
2.920014e-07 ham
[[1248, 89, 2, 84, 469, 16]]
[[1248   89    2   84  469   16    0    0    0    0    0    0    0    0
     0    0]]
1/1 [==============================] - 0s 24ms/step
0.77835506 spam
[[1, 93, 70, 2, 49, 28, 39, 261, 14, 4, 1068, 64, 663, 408]]
[[   1   93   70    2   49   28   39  261   14    4 1068   64  663  408
     0    0]]
1/1 [==============================] - 0s 24ms/step
1.4788689e-06 ham
[[92, 102, 97, 375, 230, 9, 287, 37, 8433, 18, 13, 111, 2, 345, 379]]
[[  92  102   97  375  230    9  287   37 8433   18   13  111    2  345
   379    0]]
1/1 [==============================] - 0s 26ms/step
0.99999416 spam
[[3, 17, 185, 370, 158, 16, 2, 128, 13, 152]]
[[  3  17 185 370 158  16   2 128  13 152   0   0   0   0   0   0]]
1/1 [==============================] - 0s 23ms/step
0.99999344 spam
[[79, 498, 14, 151, 98, 665, 5, 22